In [1]:
# Parsing humsavar txt file as csv
import csv
import pandas as pd
import numpy as np

In [2]:
file_path = "C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\validation\\humsavar.txt"
output_csv_path = "C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\validation\\humsavar.csv"
# 
# with open(file_path, 'r') as file:
#     # Read lines from the file
#     lines = file.readlines()
# 
# # Process each line and extract values based on column positions
# data = []
# for line in lines:
#     # Extract values based on column positions
#     column1 = line[0:11].strip()
#     column2 = line[12:22].strip()
#     column3 = line[23:34].strip()
#     column4 = line[35:49].strip()
#     column5 = line[50:58].strip()
#     column6 = line[59:73].strip()
#     column7 = line[74:].strip()
# 
#     # Append the extracted values as a list
#     data.append([column1, column2, column3, column4, column5, column6, column7])
# 
# # Open a new CSV file for writing
# with open(output_csv_path, 'w', newline='') as output_file:
#     # Create a CSV writer object
#     csv_writer = csv.writer(output_file)
#     
#     # Write the data to the new CSV file
#     csv_writer.writerows(data)

In [3]:
genes = ["GJB2", "COL2A1", "COL4A5", "COL4A3", "WFS1", "SLC26A4", "MYO7A", "FGFR1"]

# Read the CSV file into a pandas data frame object
# Leave only the rows where gene is in the list
df = pd.read_csv(output_csv_path)
df = df[df["gene"].isin(genes)]
df

,gene,uniprot_id,FTId,variant,category,dbSNP,Disease name
15766,COL2A1,P02458,VAR_001738,p.Gly267Asp,LP/P,rs121912872,Stickler syndrome 1 non-syndromic ocular (STL1...
15767,COL2A1,P02458,VAR_001739,p.Arg275Cys,LP/P,rs121912876,Czech dysplasia (CZECHD) [MIM:609162]
15768,COL2A1,P02458,VAR_001741,p.Gly303Asp,LP/P,rs121912877,Kniest dysplasia (KD) [MIM:156550]
15769,COL2A1,P02458,VAR_001742,p.Gly354Arg,LB/B,rs121912871,-
15770,COL2A1,P02458,VAR_001743,p.Gly375Arg,LP/P,-,Spondyloepiphyseal dysplasia congenital type (...
...,...,...,...,...,...,...,...
80417,WFS1,O76024,VAR_074210,p.Asp171Asn,LP/P,rs758281375,"Deafness, autosomal dominant, 6 (DFNA6) [MIM:6..."
80418,WFS1,O76024,VAR_074211,p.Tyr669His,LP/P,-,"Deafness, autosomal dominant, 6 (DFNA6) [MIM:6..."
80419,WFS1,O76024,VAR_074212,p.Arg685Pro,LP/P,rs142668478,"Deafness, autosomal dominant, 6 (DFNA6) [MIM:6..."
80420,WFS1,O76024,VAR_074213,p.His696Tyr,LP/P,-,"Deafness, autosomal dominant, 6 (DFNA6) [MIM:6..."


In [4]:
# Translate the variant name. From p.His52Arg to H52R. Use the function from the tools.
import general_tools as tools

# delete the "p." from the variant name
df["variant"] = df["variant"].apply(lambda x: x[2:])
df["variant"] = df["variant"].apply(lambda x: tools.convert_variant_to_1_letter(x))
humasavar = df
df

In [5]:
dvd_variants = pd.read_csv("C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\combined_with_source.csv")
GJB2_set = pd.read_csv("C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\gene_specific_df\\GJB2.csv")
validation_set = pd.read_csv("C:\\Users\\InbarBlech\\PycharmProjects\\Thesis\\validation\\clinvar_variants_all_genes.csv")
# change name of "Gene(s)" column to "gene"
validation_set = validation_set.rename(columns={"Gene(s)": "gene", "Protein change": "variant"})
dvd_variants = pd.concat([dvd_variants, GJB2_set])
redundancy = validation_set[validation_set.apply(lambda row: row['gene'] in dvd_variants['gene'].values and row['variant'] in dvd_variants['variant'].values, axis=1)]
# remove from validation_set, the rows that are in redundancy
validation_set = validation_set[~validation_set.apply(lambda row: row['gene'] in redundancy['gene'].values and row['variant'] in redundancy['variant'].values, axis=1)]
# concat dvd_variants and GJB2_set
complete_variants_used = pd.concat([dvd_variants, validation_set])
complete_variants_used

,gene,variant,pathogenicity,stability_WT,stability_MUT,blosum,hydrophobicity_WT,hydrophobicity_MUT,volume_WT,volume_MUT,...,sasa_MUT,RSA_WT,RSA_MUT,oda_delta,sasa_delta,pssm,entropy,source,uniprot_id,Unnamed: 0
0,MYO7A,M1H,benign,651.486,651.793,-2.0,1.9,-3.2,162.9,153.2,...,215.31,1.15335,0.996806,3.05,-18.82,5.096,2.24,patmut,NaN,NaN
1,MYO7A,M1L,benign,651.486,651.544,2.0,1.9,3.8,162.9,166.7,...,194.46,1.15335,1.018115,-0.94,-39.67,5.096,2.24,patmut,NaN,NaN
2,MYO7A,M1P,benign,651.486,651.847,-2.0,1.9,-1.6,162.9,112.7,...,177.16,1.15335,1.150390,-2.26,-56.97,5.096,2.24,patmut,NaN,NaN
3,MYO7A,M1T,benign,651.486,651.816,-1.0,1.9,-0.7,162.9,116.1,...,173.00,1.15335,1.061350,2.11,-61.13,5.096,2.24,patmut,NaN,NaN
4,MYO7A,M1W,benign,651.486,651.273,-1.0,1.9,-0.9,162.9,227.8,...,260.26,1.15335,0.985833,-11.56,26.13,5.096,2.24,patmut,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731,COL4A5,D1659E,benign,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P29400,731.0
732,COL4A5,S1659N,pathogenic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P29400,732.0
733,COL4A5,S1665N,pathogenic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P29400,733.0
734,COL4A5,T1671M,benign,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P29400,734.0


In [6]:
# Extract rows that are in both validation set and dvd variants, by comparing the gene and variant columns
redundancy = humasavar[humasavar.apply(lambda row: row['gene'] in complete_variants_used['gene'].values and row['variant'] in complete_variants_used['variant'].values, axis=1)]
redundancy

,gene,uniprot_id,FTId,variant,category,dbSNP,Disease name
15766,COL2A1,P02458,VAR_001738,G267D,LP/P,rs121912872,Stickler syndrome 1 non-syndromic ocular (STL1...
15767,COL2A1,P02458,VAR_001739,R275C,LP/P,rs121912876,Czech dysplasia (CZECHD) [MIM:609162]
15768,COL2A1,P02458,VAR_001741,G303D,LP/P,rs121912877,Kniest dysplasia (KD) [MIM:156550]
15769,COL2A1,P02458,VAR_001742,G354R,LB/B,rs121912871,-
15770,COL2A1,P02458,VAR_001743,G375R,LP/P,-,Spondyloepiphyseal dysplasia congenital type (...
...,...,...,...,...,...,...,...
80417,WFS1,O76024,VAR_074210,D171N,LP/P,rs758281375,"Deafness, autosomal dominant, 6 (DFNA6) [MIM:6..."
80418,WFS1,O76024,VAR_074211,Y669H,LP/P,-,"Deafness, autosomal dominant, 6 (DFNA6) [MIM:6..."
80419,WFS1,O76024,VAR_074212,R685P,LP/P,rs142668478,"Deafness, autosomal dominant, 6 (DFNA6) [MIM:6..."
80420,WFS1,O76024,VAR_074213,H696Y,LP/P,-,"Deafness, autosomal dominant, 6 (DFNA6) [MIM:6..."


In [7]:
# remove from validation_set, the rows that are in redundancy
humasavar = humasavar[~humasavar.apply(lambda row: row['gene'] in redundancy['gene'].values and row['variant'] in redundancy['variant'].values, axis=1)]
humasavar

,gene,uniprot_id,FTId,variant,category,dbSNP,Disease name
15783,COL2A1,P02458,VAR_001756,G1005S,LB/B,rs753342774,-
15794,COL2A1,P02458,VAR_017639,G453D,LP/P,rs794727339,Achondrogenesis 2 (ACG2) [MIM:200610]
15808,COL2A1,P02458,VAR_019836,P158L,LB/B,rs1050861,-
15814,COL2A1,P02458,VAR_023930,G855S,LP/P,rs1193507525,Spondyloepiphyseal dysplasia congenital type (...
15815,COL2A1,P02458,VAR_023931,G897V,LP/P,-,"Spondyloepimetaphyseal dysplasia, Strudwick ty..."
...,...,...,...,...,...,...,...
80398,WFS1,O76024,VAR_029500,A326V,LB/B,rs369795224,-
80401,WFS1,O76024,VAR_029506,I802V,LB/B,rs746922325,-
80408,WFS1,O76024,VAR_032967,L829P,LP/P,rs104893883,"Deafness, autosomal dominant, 6 (DFNA6) [MIM:6..."
80411,WFS1,O76024,VAR_068343,R558C,LP/P,rs199946797,Wolfram syndrome 1 (WFS1) [MIM:222300]


In [8]:
# print the genes that are in the humsavar
humasavar["gene"].unique()

array(['COL2A1', 'COL4A3', 'COL4A5', 'FGFR1', 'GJB2', 'MYO7A', 'SLC26A4',
       'WFS1'], dtype=object)

In [9]:
# Print how many rows there are for each gene in humsavar
humasavar["gene"].value_counts()

COL4A5     29
MYO7A      16
FGFR1      15
GJB2       13
SLC26A4    12
WFS1       10
COL2A1      7
COL4A3      5
Name: gene, dtype: int64